<a href="https://colab.research.google.com/github/gabrielealberto/life-insurance-simulator/blob/main/Life_insurance_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Configurazione stile matplotlib
plt.style.use('default')
sns.set_palette("husl")

In [2]:
df=pd.read_csv('mortality_table.csv')
df=df.iloc[:,0:-1]

In [3]:
columns=['Age', 'lx', 'dx', 'prob_d', 'Lx', 'Px', 'life_exp']
df.columns=columns
df['prob_d']=df['prob_d']/1000
df['dx']=df['prob_d']*df['lx']

In [4]:
df

,Age,lx,dx,prob_d,Lx,Px,life_exp
0,0,100000,257.230000,0.002572,99758,0.999743,83.417
1,1,99743,20.877207,0.000209,99732,0.999823,82.630
2,2,99722,14.405840,0.000144,99715,0.999874,81.648
3,3,99707,10.694573,0.000107,99702,0.999902,80.659
4,4,99697,8.784303,0.000088,99692,0.999913,79.668
...,...,...,...,...,...,...,...
115,115,0,0.000000,0.798995,0,0.197387,0.744
116,116,0,0.000000,0.820611,0,0.176321,0.713
117,117,0,0.000000,0.840784,0,0.156638,0.685
118,118,0,0.000000,0.859555,0,0.138300,0.660


In [5]:
age = 22
cap=400000
i=0.01
years=30

In [6]:
def y_premium(age, cap, i, years, annual_premia=False):
  mydf=df.iloc[age:age+years,:]
  start_lx=mydf.iloc[0,1]
  resdf=pd.DataFrame(columns=['year','age','probability', 'discounted_cap', 'mult'])
  resdf['age']=mydf['Age']
  resdf['probability']=mydf['dx']/start_lx
  resdf['year'] = range(1, len(resdf) + 1)
  resdf['discounted_cap']=cap/((1+i)**resdf['year'])
  resdf['mult']=resdf['discounted_cap']*resdf['probability']
  premium=resdf['mult'].sum()
  mydf = mydf.reset_index(drop=True)

  if annual_premia:
    ratz=pd.DataFrame(columns=['year', 'L_probability'])
    start_Lx=mydf.iloc[0,4]

    ratz['year']=range(0, len(resdf))
    ratz['L_probability']=mydf['Lx']/start_Lx

    ratz['discount_factor']=1/((1+i)**ratz['year'])
    ratz['mult']=ratz['L_probability']*ratz['discount_factor']
    denominator=ratz['mult'].sum()
    annual_premium=premium/denominator



  return resdf, ratz, annual_premium, annual_premium/12

In [7]:
results=y_premium(age, cap, i, years, annual_premia=True)

In [8]:
def create_insurance_dashboard(annual_premium, monthly_premium):
    """
    Create a simplified dashboard for insurance premium analysis
    """

    print("🎯" + "="*80)
    print("📊 LIFE TCM INSURANCE PREMIUM DASHBOARD")
    print("🎯" + "="*80)
    print(f"👤 Insured: {age} years | 💰 Capital: €{cap:,} | ⏱️ Duration: {years} years")
    print("="*82)

    # ============================================================================
    # MAIN STATISTICS SECTION (PREMIUMS ONLY)
    # ============================================================================

    # Create the box with annual and monthly premium only
    stats_html = f"""
        <div style="
            background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
            padding: 30px;
            border-radius: 20px;
            margin: 30px auto;
            max-width: 700px;
            color: white;
            box-shadow: 0 15px 35px rgba(0,0,0,0.25);
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        ">
            <h2 style="
                text-align: center;
                margin-bottom: 25px;
                font-size: 26px;
                letter-spacing: 1px;
                color: white;
            ">
                💼 Premium Calculation Results
            </h2>
            <div style="
                display: grid;
                grid-template-columns: 1fr 1fr;
                gap: 30px;
                text-align: center;
            ">
                <div style="
                    background: rgba(255,255,255,0.12);
                    padding: 30px;
                    border-radius: 20px;
                    backdrop-filter: blur(12px);
                    border: 1.5px solid rgba(255,255,255,0.25);
                    transition: transform 0.3s ease, box-shadow 0.3s ease;
                " onmouseover="this.style.transform='translateY(-5px)'; this.style.boxShadow='0 20px 40px rgba(0,0,0,0.3)';"
                  onmouseout="this.style.transform='translateY(0)'; this.style.boxShadow='0 15px 35px rgba(0,0,0,0.25)';">
                    <h3 style="margin-bottom: 15px; font-size: 20px; color: white;">🎯 Annual Premium</h3>
                    <h2 style="color: #4ade80; margin: 20px 0; font-size: 32px;">€{annual_premium:,.2f}</h2>
                    <p style="margin-top: 10px; font-size: 15px; opacity: 0.85; color: white;">Pure annual premium</p>
                </div>
                <div style="
                    background: rgba(255,255,255,0.12);
                    padding: 30px;
                    border-radius: 20px;
                    backdrop-filter: blur(12px);
                    border: 1.5px solid rgba(255,255,255,0.25);
                    transition: transform 0.3s ease, box-shadow 0.3s ease;
                " onmouseover="this.style.transform='translateY(-5px)'; this.style.boxShadow='0 20px 40px rgba(0,0,0,0.3)';"
                  onmouseout="this.style.transform='translateY(0)'; this.style.boxShadow='0 15px 35px rgba(0,0,0,0.25)';">
                    <h3 style="margin-bottom: 15px; font-size: 20px; color: white;">📅 Monthly Premium</h3>
                    <h2 style="color: #60a5fa; margin: 20px 0; font-size: 32px;">€{monthly_premium:,.2f}</h2>
                    <p style="margin-top: 10px; font-size: 15px; opacity: 0.85; color: white;">Equivalent monthly installment</p>
                </div>
            </div>
        </div>
    """
    display(HTML(stats_html))

result = create_insurance_dashboard(results[2], results[3])


🎯================================================================================
📊 LIFE TCM INSURANCE PREMIUM DASHBOARD
🎯================================================================================
👤 Insured: 22 years | 💰 Capital: €400,000 | ⏱️ Duration: 30 years
